In [1]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.chat_models import ChatOllama
import pandas as pd
import json
import re

## 提取出文本中的物品

In [2]:
# Temp = 0 so that we get clean information without a lot of creativity
chat_model = ChatOllama(temperature=0, model="qwen:32b-chat-v1.5-q5_K_M")

text_input = """
在这个宽敞的维多利亚风格卧室中，家具和陈设的精致细节彰显了19世纪的优雅与奢华。房间的中心是一张四柱大床，床头板雕刻着复杂的花卉图案，床尾则装饰着流苏的床尾凳，
增添了一丝贵族气息。床的两侧，是一对精美的床头柜，上面摆放着镀金的台灯和小巧的时钟，细节之处透露着维多利亚时代的精致。
房间的窗户朝南，阳光透过蕾丝窗帘洒入室内，为整个空间带来了温暖和明亮。窗户下方，一张软垫的长椅摆放在那里，是阅读或沉思的理想之地。
对面的墙上，是一面装饰着精美雕花的壁炉，即使在不使用的季节，它也是房间内不可或缺的装饰元素。
在房间的一角，摆放着一个小巧的书桌和书架，书架上整齐地排列着各种书籍和装饰品，书桌上则摆放着精致的文具和一盏台灯，为这个空间增添了一份学术氛围。
整个房间的布局既实用又美观，反映了维多利亚时代对于家居美学的追求。
"""

get_object_chain = ChatPromptTemplate.from_template("提取出如下文本中的陈设、家具和物品，输出一个数组，尽量详尽。文本：{text_input}") | chat_model | StrOutputParser()

objects_in_text = get_object_chain.invoke({"text_input": text_input})

print(objects_in_text)

['四柱大床', '床头板', '流苏的床尾凳', '床头柜', '镀金的台灯', '小巧的时钟', '窗户', '蕾丝窗帘', '软垫的长椅', '壁炉', '书桌', '书架', '书籍', '装饰品', '精致的文具', '台灯']


## 在素材库中找到匹配的物品

In [3]:
# How you would like your response structured. This is basically a fancy prompt template
response_schemas = [
    ResponseSchema(name="object_in_text", description="这是用户输入的物品"),
    ResponseSchema(name="object_in_gallery", description="这是你觉得与用户输入最匹配的物品"),
    ResponseSchema(name="match_score", description="你认为用户输入的匹配与你匹配的接近程度的0-100分")
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# See the prompt template you created for formatting
format_instructions = output_parser.get_format_instructions()
print (output_parser.get_format_instructions())

template = """
您将获得用户提供的一系列物品名称。
在已入库物品列表中找到最合适的匹配项。
最接近的匹配将是具有最接近语义的匹配。不仅仅是字符串相似。

{format_instructions}

用左括号和右括号包装你的最终输出(一个json对象列表)

用户输入的物品:
{objects_in_text}

已入库物品:
{objects_name_in_gallery}

你的反应:
"""

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(template)  
    ],
    input_variables=["object_in_text", "objects_in_gallery"],
    partial_variables={"format_instructions": format_instructions}
)

def GetObjectsInGallery(csv_file_path):
    df = pd.read_csv(csv_file_path)
    objects = df['物品'].values
    objects_url = df['URL'].values
    objects_x_size = df['X轴尺寸'].values
    objects_y_size = df['Y轴尺寸'].values
    objects_z_size = df['Z轴尺寸'].values
    objects_scale = df['Scale'].values

    objects_in_gallery = []
    for i in range(len(objects)):
        obj = {'object_in_gallery':objects[i], 'objects_url_in_gallery':objects_url[i], 
        'objects_x_size':objects_x_size[i], 'objects_y_size':objects_y_size[i], 'objects_z_size':objects_z_size[i], 'objects_scale':objects_scale[i]}
        objects_in_gallery.append(obj)

    return objects_in_gallery

def GetObjectsNameInGallery(csv_file_path):
    df = pd.read_csv(csv_file_path)
    objects = ", ".join(df['物品'].values)
    return objects

objects_name_in_gallery = GetObjectsNameInGallery('../data/ObjectsInGallery_LayoutTest.csv')

# Your user input
find_objects_input = prompt.format_prompt(objects_in_text=objects_in_text, objects_name_in_gallery=objects_name_in_gallery)

# print (f"There are {len(_input.messages)} message(s)")
# print (f"Type: {type(_input.messages[0])}")
# print ("---------------------------")
# print (find_objects_input.messages[0].content)

find_objects_output = chat_model(find_objects_input.to_messages())

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"object_in_text": string  // 这是用户输入的物品
	"object_in_gallery": string  // 这是你觉得与用户输入最匹配的物品
	"match_score": string  // 你认为用户输入的匹配与你匹配的接近程度的0-100分
}
```


/home/huangwei1/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [4]:
# 提取JSON内容
def StripJSONString(output):
    if "```json" in output.content:
        json_string = output.content.split("```json")[1].strip()
    else:
        json_string = output.content

    if "```" in json_string:
        json_string = json_string.split("```")[0].strip()


    # 去掉不符合格式的内容
    # 使用正则表达式替换掉 // 和其后面的所有内容，直到行尾
    cleaned_json_str = re.sub(r'//.*', '', json_string)
    return cleaned_json_str




# output_parser.parse(output.content) Ideally this works but not in all cases
objects_found = json.loads(StripJSONString(find_objects_output))
objects_in_gallery = GetObjectsInGallery('../data/ObjectsInGallery_LayoutTest.csv')
print(objects_in_gallery)

objects_matched = []
for obj in objects_found:
    if int(obj['match_score']) < 80:
        continue

    for obj_g in objects_in_gallery:
        if obj_g['object_in_gallery'] == obj['object_in_gallery']:
            obj.update(obj_g)
            objects_matched.append(obj)
            break

print(objects_matched)

# 存储为JSON文件
with open("../data/ObjectsMatched.json", 'w') as f:
    json.dump(objects_matched, f, default=int, ensure_ascii=False, indent=4)

# 表格展示
pd.DataFrame(objects_matched)

[{'object_in_gallery': '仿古灯', 'objects_url_in_gallery': 'D:/Home/Downloads/LayoutTestObjects/Antique_Lamp.usdz', 'objects_x_size': 50, 'objects_y_size': 150, 'objects_z_size': 50, 'objects_scale': 1.0}, {'object_in_gallery': '古董吊灯', 'objects_url_in_gallery': 'D:/Home/Downloads/LayoutTestObjects/Antique_Chandelier.usdz', 'objects_x_size': 100, 'objects_y_size': 200, 'objects_z_size': 100, 'objects_scale': 1.0}, {'object_in_gallery': '古董木制书柜', 'objects_url_in_gallery': 'D:/Home/Downloads/LayoutTestObjects/Antique_wooden_bookcase_-_Game_model.usdz', 'objects_x_size': 200, 'objects_y_size': 200, 'objects_z_size': 40, 'objects_scale': 1.0}, {'object_in_gallery': '圆桌和椅子', 'objects_url_in_gallery': 'D:/Home/Downloads/LayoutTestObjects/Round_Table_And_Chairs.usdz', 'objects_x_size': 300, 'objects_y_size': 100, 'objects_z_size': 300, 'objects_scale': 1.0}, {'object_in_gallery': '吊灯', 'objects_url_in_gallery': 'D:/Home/Downloads/LayoutTestObjects/Lamps.usdz', 'objects_x_size': 150, 'objects_y_si

,object_in_text,object_in_gallery,match_score,objects_url_in_gallery,objects_x_size,objects_y_size,objects_z_size,objects_scale
0,四柱大床,维多利亚风格床,85,D:/Home/Downloads/LayoutTestObjects/Victorian_...,150,100,200,1.0
1,床头柜,维多利亚风格床头柜,90,D:/Home/Downloads/LayoutTestObjects/Bedside_ta...,40,80,40,4.0
2,镀金的台灯,维多利亚风格台灯,80,D:/Home/Downloads/LayoutTestObjects/Victorian_...,10,12,10,10.0
3,书桌,维多利亚风格书桌,85,D:/Home/Downloads/LayoutTestObjects/Game_Ready...,100,60,50,1.0
4,台灯,维多利亚风格台灯,85,D:/Home/Downloads/LayoutTestObjects/Victorian_...,10,12,10,10.0


## 根据文本确定物品方位和详细的相对位置

In [8]:
# 使用self-discover方法
from langchain import hub

select_prompt = hub.pull("superfhwl/self-discovery-select")
adapt_prompt = hub.pull("superfhwl/self-discovery-adapt")
structured_prompt = hub.pull("superfhwl/self-discovery-structure")
reasoning_prompt = hub.pull("superfhwl/self-discovery-reasoning")

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

select_chain = select_prompt | chat_model | StrOutputParser()
adapt_chain = adapt_prompt | chat_model | StrOutputParser()
structure_chain = structured_prompt | chat_model | StrOutputParser()
reasoning_chain = reasoning_prompt | chat_model | StrOutputParser()
overall_chain = (
    RunnablePassthrough.assign(selected_modules=select_chain)
    .assign(adapted_modules=adapt_chain)
    .assign(reasoning_structure=structure_chain)
    .assign(answer=reasoning_chain)
)

# with open("../data/ReasoningModules.json", 'r') as f:
#     reasoning_modules = json.load(f)

reasoning_modules = [
    "1. 我怎样才能设计一个实验来解决这个问题呢?"
    "2. 把解决这个问题的想法列一个清单，然后一个接一个地应用到这个问题上，看看是否能取得进展. "
    "3. 我怎样才能衡量这个问题的进展呢?"
    "4. 我怎样才能简化这个问题，使它更容易解决?"
    "5. 这个问题背后的关键假设是什么?"
    # "6. 每种解决方案的潜在风险和缺点是什么?"
    # "7. 对这个问题有什么不同的观点或观点?"
    # "8. 这个问题及其解决方案的长期影响是什么?"
    "9. 我怎样才能把这个问题分解成更小、更容易处理的部分呢?"
    # "10. 批判性思维:这种风格包括从不同的角度分析问题，质疑假设，评估现有的证据或信息. 它侧重于逻辑推理、基于证据的决策，以及识别思维中的潜在偏见或缺陷. "
    # "11. 尝试创造性思维，产生创新的和打破常规的想法来解决问题. 探索非传统的解决方案，超越传统的界限思考，鼓励想象力和独创性. "
    # "12. 寻求他人的意见和合作来解决问题. 强调团队合作，开放沟通，利用团队的不同观点和专业知识来提出有效的解决方案. "
    # "13. 使用系统思维:将问题视为更大系统的一部分，并理解各种元素的相互联系. 重点是确定影响问题的潜在原因、反馈回路和相互依赖关系，并制定整体解决方案，以解决整个系统. "
    # "14. 使用风险分析:评估与问题的不同解决方案或方法相关的潜在风险、不确定性和权衡. 强调评估潜在的后果和成功或失败的可能性，并在平衡分析风险和收益的基础上做出明智的决定. "
    # "15. 运用反思性思维:从问题中退后一步，花时间反省和自我反省. 检查可能影响解决问题的个人偏见、假设和心理模型，并从过去的经验中学习，以改进未来的方法. "
    "16. 需要解决的核心问题是什么?"
    "17. 分析平面布局需要哪方面的专业知识？"
    "18. 坐标和朝向的规律是怎么样的？"
    "19. 用户给出的物品是哪些？他们的尺寸是怎样的？"
    "20. 物品之间发生碰撞了吗？如何检查？"
    "21. 物品的布局符合生活便利性了吗？间距是否合理？如果没有，要如何调整？"
    "22. 物品的正面朝向是否正确？紧靠墙壁的物品，是背靠墙还是面向墙的？"
    # "17. 造成这个问题的根本原因或因素是什么?"
    # "18. 有没有什么潜在的解决方案或策略是以前尝试过的?如果是，结果和教训是什么?"
    # "19. 在解决这个问题的过程中可能会遇到哪些潜在的障碍或挑战?"
    # "20. 是否有任何相关的数据或信息，可以提供洞察问题?如果是，有哪些可用的数据来源，如何分析这些数据?"
    # "21. 是否有任何利益相关者或个人直接受到这个问题的影响?他们的观点和需求是什么?"
    # "22. 需要什么资源(财政、人力、技术等)才能有效地解决这个问题?"
    # "23. 如何衡量或评估解决问题的进展或成功?"
    # "24. 可以使用哪些指标或度量标准?"
    # "25. 问题是技术性的还是实践性的，需要特定的专业知识或技能?或者它更像是一个概念或理论问题?"
    # "26. 问题是否涉及物理限制，例如有限的资源、基础设施或空间?"
    # "27. 这个问题是否与人类行为有关，比如社会、文化或心理问题?"
    # "28. 问题是否涉及决策或计划，需要在不确定或目标相互竞争的情况下做出选择?"
    # "29. 这个问题是需要数据分析、建模或优化技术的分析性问题吗?"
    # "30. 这个问题是一个需要创造性解决方案和创新的设计挑战吗?"
    # "31. 这个问题是否需要解决系统或结构性问题，而不仅仅是个别问题?"
    # "32. 这个问题是时间敏感还是紧急，需要立即注意和采取行动?"
    # "33. 对于这类问题说明，通常会产生什么样的解决方案?"
    # "34. 根据问题说明和当前的最佳解决方案，猜测其他可能的解决方案. "
    # "35. 让我们假设当前的最佳解决方案是完全错误的，还有什么其他方式来思考问题规范?"
    # "36. 根据您对这类问题说明的了解，修改当前最佳解决方案的最佳方法是什么?"
    # "37. 忽略当前的最佳解决方案，创造一个全新的解决方案. "
    "38. 让我们一步一步来思考. "
    "39. 让我们制定一个循序渐进的计划，并用好的符号和解释来实施它. "
]

reasoning_modules_str = "\n".join(reasoning_modules)


# 读取JSON数据
with open('../data/ObjectsMatched.json', 'r') as f:
    objects_matched = json.load(f)

# 根据匹配信息整理输入布局的数据
user_input_objects = []
for obj in objects_matched:
    if int(obj['match_score']) > 0:
        obj_mat = {'物品名称':obj['object_in_text'], 
                    '长度':obj['objects_x_size'], 
                    '高度':obj['objects_y_size'], 
                    '宽度':obj['objects_z_size']}
        user_input_objects.append(obj_mat)


task = f"""
您将获得用户提供的一段文本，描述了房间中的物品和陈设。同时还有一组用户指定的物品名称，它们与文本中的物品和陈设对应。同时还有物品的 X、Y、Z轴尺寸，以厘米为单位。
分析并输出一个物品和陈设的布局方案，应符合文本描述的内容。同时要符合人们日常生活的习惯，考虑生活便利性。物品和物品之间不要出现重叠和碰撞，间距和朝向要合理。
X轴指向东、Y轴指向上、Z轴指向南。场景中心坐标为 X=0,Y=0,Z=0。场景中心朝向为南，场景尺寸为X=800,Y=500,Z=800。物品朝向的角度：南=0、东=90、北=180、西=270
布局方案采用JSON格式输出，包含：用户指定的物品名称、俯视图位置(X、Z轴坐标)、朝向的角度

用户指定的物品信息:
{user_input_objects}

用户提供的一段文本:
{text_input}
"""

layout_output = overall_chain.invoke(
    {"task_description": task, "reasoning_modules": reasoning_modules_str}
)

for k in layout_output:
    print(k)
    print('==============')
    print(layout_output[k])

task_description

您将获得用户提供的一段文本，描述了房间中的物品和陈设。同时还有一组用户指定的物品名称，它们与文本中的物品和陈设对应。同时还有物品的 X、Y、Z轴尺寸，以厘米为单位。
分析并输出一个物品和陈设的布局方案，应符合文本描述的内容。同时要符合人们日常生活的习惯，考虑生活便利性。物品和物品之间不要出现重叠和碰撞，间距和朝向要合理。
X轴指向东、Y轴指向上、Z轴指向南。场景中心坐标为 X=0,Y=0,Z=0。场景中心朝向为南，场景尺寸为X=800,Y=500,Z=800。物品朝向的角度：南=0、东=90、北=180、西=270
布局方案采用JSON格式输出，包含：用户指定的物品名称、俯视图位置(X、Z轴坐标)、朝向的角度

用户指定的物品信息:
[{'物品名称': '四柱大床', '长度': 150, '高度': 100, '宽度': 200}, {'物品名称': '床头柜', '长度': 40, '高度': 80, '宽度': 40}, {'物品名称': '镀金的台灯', '长度': 10, '高度': 12, '宽度': 10}, {'物品名称': '书桌', '长度': 100, '高度': 60, '宽度': 50}, {'物品名称': '台灯', '长度': 10, '高度': 12, '宽度': 10}]

用户提供的一段文本:

在这个宽敞的维多利亚风格卧室中，家具和陈设的精致细节彰显了19世纪的优雅与奢华。房间的中心是一张四柱大床，床头板雕刻着复杂的花卉图案，床尾则装饰着流苏的床尾凳，
增添了一丝贵族气息。床的两侧，是一对精美的床头柜，上面摆放着镀金的台灯和小巧的时钟，细节之处透露着维多利亚时代的精致。
房间的窗户朝南，阳光透过蕾丝窗帘洒入室内，为整个空间带来了温暖和明亮。窗户下方，一张软垫的长椅摆放在那里，是阅读或沉思的理想之地。
对面的墙上，是一面装饰着精美雕花的壁炉，即使在不使用的季节，它也是房间内不可或缺的装饰元素。
在房间的一角，摆放着一个小巧的书桌和书架，书架上整齐地排列着各种书籍和装饰品，书桌上则摆放着精致的文具和一盏台灯，为这个空间增添了一份学术氛围。
整个房间的布局既实用又美观，反映了维多利亚时代对于家居美学的追求。


reasoning_modules
1. 我怎样才能设计一个实验来解决这个问题呢?2. 

## 根据方位和相对位置，转换为坐标

In [9]:

response_schemas = [
    ResponseSchema(name="object_in_text", description="这是用户输入的物品名称"),
    ResponseSchema(name="object_name", description="这是用户输入的物品的英语翻译（合法的C语言变量名称形式），使用_1这样的格式避免重名"),
    ResponseSchema(name="translation_x", description="这是物品的位置X轴坐标，单位是厘米"),
    # ResponseSchema(name="translation_y", description="这是物品的位置Y轴坐标，固定为0"),
    ResponseSchema(name="translation_z", description="这是物品的位置Z轴坐标，单位是厘米"),
    ResponseSchema(name="rotation_y", description="这是朝向角度（南=0、东=90、北=180、西=270）")
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# See the prompt template you created for formatting
format_instructions = output_parser.get_format_instructions()
print (output_parser.get_format_instructions())

template = """
您将获得一段用户输入的文本，描述了物品的位置(X、Z轴坐标)，朝向的角度（以下数字之一：0、90、180、270）。

根据上述信息输出：

{format_instructions}

用左括号和右括号包装你的最终输出(一个json对象列表)

用户输入的物品信息:
{text_input}

你的反应:
"""

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(template)  
    ],
    input_variables=["text_input"],
    partial_variables={"format_instructions": format_instructions}
)

# Your user input
layout_input = prompt.format_prompt(text_input=layout_output['answer'])


print (f"There are {len(layout_input.messages)} message(s)")
print (f"Type: {type(layout_input.messages[0])}")
print ("---------------------------")
print (layout_input.messages[0].content)

layout_formated_output = chat_model(layout_input.to_messages())

# output_parser.parse(output.content) Ideally this works but not in all cases
layout_formated = json.loads(StripJSONString(layout_formated_output))

# 存储为JSON文件
with open("../data/ObjectsLayout.json", 'w') as f:
    json.dump(layout_formated, f, ensure_ascii=False, indent=4)

# 表格展示
pd.DataFrame(layout_formated)


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"object_in_text": string  // 这是用户输入的物品名称
	"object_name": string  // 这是用户输入的物品的英语翻译（合法的C语言变量名称形式），使用_1这样的格式避免重名
	"translation_x": string  // 这是物品的位置X轴坐标，单位是厘米
	"translation_z": string  // 这是物品的位置Z轴坐标，单位是厘米
	"rotation_y": string  // 这是朝向角度（南=0、东=90、北=180、西=270）
}
```
There are 1 message(s)
Type: <class 'langchain_core.messages.human.HumanMessage'>
---------------------------

您将获得一段用户输入的文本，描述了物品的位置(X、Z轴坐标)，朝向的角度（以下数字之一：0、90、180、270）。

根据上述信息输出：

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"object_in_text": string  // 这是用户输入的物品名称
	"object_name": string  // 这是用户输入的物品的英语翻译（合法的C语言变量名称形式），使用_1这样的格式避免重名
	"translation_x": string  // 这是物品的位置X轴坐标，单位是厘米
	"translation_z": string  // 这是物品的位置Z轴坐标，单位是厘米
	"rotation_y": string  // 这是朝向角度（南=0、东=90、北=180、西=2

,object_in_text,object_name,translation_x,translation_z,rotation_y
0,四柱大床,four_poster_bed,350,250,0
1,床头柜,nightstand_1,410,250,90
2,床头柜,nightstand_2,390,250,90
3,镀金的台灯,gilded_lamp_1,415,255,90
4,镀金的台灯,gilded_lamp_2,385,255,90
5,窗户,window,700,250,180
6,长椅,sofa,660,300,180
7,壁炉,fireplace,100,250,0
8,书桌,desk,150,400,270
9,台灯,lamp,165,405,270


## 将JSON文件转换为USDA文件

In [10]:
import json
import os
from pxr import Usd, UsdGeom, Sdf, Gf

# 定义一个函数来创建USD对象
def create_usd_object(stage, prim_path, obj_name, obj_url, position, rotation):
    # 创建一个新的Prim在给定的路径上
    prim = stage.DefinePrim(prim_path, 'Xform')
    prim.name = obj_name

    xformable = UsdGeom.Xformable(prim)

    xformable.AddTranslateOp().Set(value=translation)
    xformable.AddRotateXYZOp().Set(value=rotation)

    # 创建payload，并设置其references指向目标USD文件
    payload = Sdf.Payload(obj_url)
    prim.GetPayloads().AddPayload(payload)

    print(xformable)
    
    return prim

from datetime import datetime

current_time = datetime.now()
formatted_time = current_time.strftime('%Y-%m-%d %H:%M:%S')
print(formatted_time)

# 创建USD stage
stage = Usd.Stage.CreateNew("../data/LayoutTest_" + formatted_time + ".usda")

# 创建一个新的Prim，这将作为默认Prim
default_prim = stage.DefinePrim('/Objects')

# 使用SetDefaultPrim方法设置默认Prim
stage.SetDefaultPrim(default_prim)

# 读取JSON数据
with open('../data/ObjectsMatched.json', 'r') as f:
    objects_matched = json.load(f)

with open('../data/ObjectsLayout.json', 'r') as f:
    layout_formated = json.load(f)

# # 合并两个JSON文件的数据
for obj in objects_matched:
    if int(obj['match_score']) <= 0:
        continue

    for obj_layout in layout_formated:
        if obj_layout['object_in_text'] == obj['object_in_text']:
            obj_layout.update(obj)

# for o in ObjectsLayout:
#     print(o)

# for o in ObjectsMatched:
#     print(o)

# 遍历JSON数据，为每个对象创建一个USD prim
for obj in layout_formated:
    if not 'objects_url_in_gallery' in obj:
        continue

    prim_path = '/Objects/' + obj['object_name']
    # translation = Gf.Vec3f(float(obj['translation_x']), float(obj['translation_y']), float(obj['translation_z']))
    translation = Gf.Vec3f(float(obj['translation_x']), 0, float(obj['translation_z']))
    rotation = Gf.Vec3f(0, float(obj['rotation_y']), 0)
    create_usd_object(stage, prim_path, obj['object_in_text'], obj['objects_url_in_gallery'], translation, rotation)

# 保存USD stage
stage.Save()


2024-05-04 01:02:48
UsdGeom.Xformable(Usd.Prim(</Objects/four_poster_bed>))
UsdGeom.Xformable(Usd.Prim(</Objects/nightstand_1>))
UsdGeom.Xformable(Usd.Prim(</Objects/nightstand_2>))
UsdGeom.Xformable(Usd.Prim(</Objects/gilded_lamp_1>))
UsdGeom.Xformable(Usd.Prim(</Objects/gilded_lamp_2>))
UsdGeom.Xformable(Usd.Prim(</Objects/desk>))
UsdGeom.Xformable(Usd.Prim(</Objects/lamp>))
